In [83]:
import soundfile
import numpy as np
import librosa
import glob
import os
from sklearn.model_selection import train_test_split
from pydub import AudioSegment

# all emotions on RAVDESS dataset
int2emotion = {
    "01": "neutral",
    "02": "calm",
    "03": "happy",
    "04": "sad",
    "05": "angry",
    "06": "fearful",
    "07": "disgust",
    "08": "surprised"
}

# we allow only these emotions
AVAILABLE_EMOTIONS = {
    "angry",
    "sad",
    "neutral",
    "happy",
    "calm",
    "fearful"
}

In [84]:
def extract_feature(file_name, **kwargs):
    """
    Extract feature from audio file `file_name`
        Features supported:
            - MFCC (mfcc)
            - Chroma (chroma)
            - MEL Spectrogram Frequency (mel)
            - Contrast (contrast)
            - Tonnetz (tonnetz)
        e.g:
        `features = extract_feature(path, mel=True, mfcc=True)`
    """
    mfcc = kwargs.get("mfcc")
    chroma = kwargs.get("chroma")
    mel = kwargs.get("mel")
    contrast = kwargs.get("contrast")
    tonnetz = kwargs.get("tonnetz")
    with soundfile.SoundFile(file_name) as sound_file:
        X = sound_file.read(dtype="float32")
        sample_rate = sound_file.samplerate
        if chroma or contrast:
            stft = np.abs(librosa.stft(X))
        result = np.array([])
        if mfcc:
            mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
            result = np.hstack((result, mfccs))
        if chroma:
            chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
            result = np.hstack((result, chroma))
        if mel:
            mel = np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
            result = np.hstack((result, mel))
        if contrast:
            contrast = np.mean(librosa.feature.spectral_contrast(S=stft, sr=sample_rate).T,axis=0)
            result = np.hstack((result, contrast))
        if tonnetz:
            tonnetz = np.mean(librosa.feature.tonnetz(y=librosa.effects.harmonic(X), sr=sample_rate).T,axis=0)
            result = np.hstack((result, tonnetz))
    return result

In [85]:
def load_data(test_size=0.2):
    X, y = [], []
    
    for file in glob.glob("/content/drive/MyDrive/Ravadess speech & song/Actor_*/*.wav"):
        # get the base name of the audio file
        basename = os.path.basename(file)
        sound = AudioSegment.from_wav(file)
        sound = sound.set_channels(1)
        sound.export(file, format="wav")
        print(basename)
        # get the emotion label
        emotion = int2emotion[basename.split("-")[2]]
        # we allow only AVAILABLE_EMOTIONS we set
        if emotion not in AVAILABLE_EMOTIONS:
            continue
        # extract speech features
        features = extract_feature(file, mfcc=True, chroma=False, mel=False)
        # add to data
        X.append(features)
        l={'happy':0.0,'sad':1.0,'neutral':3.0,'angry':4.0,'calm':5.0,'fearful':6.0}
        y.append(l[emotion])
    
    # split the data to training and testing and return it
    return train_test_split(np.array(X), y, test_size=test_size, random_state=7)

In [86]:
X_train, X_test, y_train, y_test = load_data(test_size=0.25)
X_train , X_val , y_train , y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=7)
print("[+] Number of training samples:", X_train.shape)
# number of samples in testing data
print("[+] Number of testing samples:", X_test.shape)
print("[+] Number of validation samples:", X_val.shape)

03-02-02-02-02-01-01.wav
03-02-03-01-01-02-01.wav
03-02-02-02-01-02-01.wav
03-02-02-01-02-01-01.wav
03-02-03-01-01-01-01.wav
03-02-01-01-02-01-01.wav
03-02-02-02-01-01-01.wav
03-02-03-01-02-01-01.wav
03-02-03-01-02-02-01.wav
03-02-02-01-01-02-01.wav
03-02-01-01-01-01-01.wav
03-02-01-01-02-02-01.wav
03-02-02-01-01-01-01.wav
03-02-01-01-01-02-01.wav
03-02-05-02-01-01-01.wav
03-02-02-02-02-02-01.wav
03-02-05-01-02-01-01.wav
03-02-04-02-01-02-01.wav
03-02-05-01-01-02-01.wav
03-02-04-01-01-02-01.wav
03-02-02-01-02-02-01.wav
03-02-05-02-01-02-01.wav
03-02-04-01-01-01-01.wav
03-02-03-02-02-01-01.wav
03-02-05-01-02-02-01.wav
03-02-04-01-02-02-01.wav
03-02-04-02-02-02-01.wav
03-02-05-01-01-01-01.wav
03-02-03-02-02-02-01.wav
03-02-04-01-02-01-01.wav
03-02-04-02-01-01-01.wav
03-02-04-02-02-01-01.wav
03-02-03-02-01-01-01.wav
03-02-03-02-01-02-01.wav
03-02-05-02-02-01-01.wav
03-02-06-02-02-02-01.wav
03-02-06-01-02-01-01.wav
03-02-06-01-01-02-01.wav
03-02-06-02-02-01-01.wav
03-02-05-02-02-02-01.wav


In [87]:
import numpy as np
X_train = np.asarray(X_train)
y_train= np.asarray(y_train)
X_val= np.asarray(X_val)
y_val= np.asarray(y_val)
X_test=np.array(X_test)
y_test=np.array(y_test)

In [88]:
X_train.shape,y_train.shape,X_test.shape,y_test.shape , X_val.shape ,y_val.shape

((1240, 40), (1240,), (517, 40), (517,), (311, 40), (311,))

In [89]:
x_traincnn = np.expand_dims(X_train, axis=2)
x_valcnn = np.expand_dims(X_val, axis=2)
x_testcnn = np.expand_dims(X_test, axis=2)


In [90]:
x_traincnn.shape,x_testcnn.shape,x_valcnn.shape

((1240, 40, 1), (517, 40, 1), (311, 40, 1))

In [101]:
import keras
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from tensorflow.keras.utils import to_categorical
from keras.layers import Input, Flatten, Dropout, Activation
from keras.layers import Conv1D, MaxPooling1D
from keras.models import Model
from keras.callbacks import ModelCheckpoint
from tensorflow.keras import optimizers

model = Sequential()

model.add(Conv1D(128, 5,padding='same',input_shape=(40,1)))        #1
model.add(Activation('relu'))
model.add(Dropout(0.1))
model.add(MaxPooling1D(pool_size=(8)))



model.add(Conv1D(128, 5,padding='same',))                           #2
model.add(Activation('relu'))
model.add(Dropout(0.1))

model.add(Flatten())
model.add(Dense(8))                                                 #3
model.add(Activation('softmax'))
opt = optimizers.RMSprop(lr=0.00005, rho=0.9, epsilon=None, decay=0.0)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


In [102]:
model.summary()


Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_18 (Conv1D)          (None, 40, 128)           768       
                                                                 
 activation_27 (Activation)  (None, 40, 128)           0         
                                                                 
 dropout_18 (Dropout)        (None, 40, 128)           0         
                                                                 
 max_pooling1d_9 (MaxPooling  (None, 5, 128)           0         
 1D)                                                             
                                                                 
 conv1d_19 (Conv1D)          (None, 5, 128)            82048     
                                                                 
 activation_28 (Activation)  (None, 5, 128)            0         
                                                      

In [103]:
model.compile(loss='sparse_categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

In [104]:
cnnhistory=model.fit(x_traincnn, y_train, batch_size=4, epochs=500, validation_data=(x_valcnn, y_val))


Epoch 1/500
310/310 [==============================] - 2s 5ms/step - loss: 6.4741 - accuracy: 0.1823 - val_loss: 3.2562 - val_accuracy: 0.1961
Epoch 2/500
310/310 [==============================] - 1s 5ms/step - loss: 5.2152 - accuracy: 0.1944 - val_loss: 1.9940 - val_accuracy: 0.1865
Epoch 3/500
310/310 [==============================] - 1s 5ms/step - loss: 4.2672 - accuracy: 0.1895 - val_loss: 2.0680 - val_accuracy: 0.2058
Epoch 4/500
310/310 [==============================] - 1s 5ms/step - loss: 3.5167 - accuracy: 0.2089 - val_loss: 1.8280 - val_accuracy: 0.2283
Epoch 5/500
310/310 [==============================] - 1s 5ms/step - loss: 2.9683 - accuracy: 0.2113 - val_loss: 1.8666 - val_accuracy: 0.2283
Epoch 6/500
310/310 [==============================] - 2s 5ms/step - loss: 2.4930 - accuracy: 0.2347 - val_loss: 2.0805 - val_accuracy: 0.1994
Epoch 7/500
310/310 [==============================] - 2s 5ms/step - loss: 2.3082 - accuracy: 0.2395 - val_loss: 1.6612 - val_accuracy: 0.3119

In [105]:
em=['happy','sad','neutral','angry','calm','fearful']


In [106]:
loss, acc = model.evaluate(x_testcnn, y_test)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

17/17 [==============================] - 0s 3ms/step - loss: 0.9014 - accuracy: 0.7485
Restored model, accuracy: 74.85%


In [107]:
from sklearn.metrics import classification_report 
from sklearn.metrics import confusion_matrix 
predictions = model.predict(x_testcnn)
classes_x=np.argmax(predictions,axis=1)
print(classification_report(y_test,classes_x))
print(confusion_matrix(y_test,classes_x))

              precision    recall  f1-score   support

         0.0       0.80      0.73      0.76        99
         1.0       0.73      0.68      0.71       101
         3.0       0.64      0.78      0.71        46
         4.0       0.77      0.85      0.80        85
         5.0       0.75      0.81      0.78        94
         6.0       0.76      0.67      0.71        92

    accuracy                           0.75       517
   macro avg       0.74      0.75      0.75       517
weighted avg       0.75      0.75      0.75       517

[[72  2  3  8 10  4]
 [ 6 69  9  2  7  8]
 [ 0  2 36  0  8  0]
 [ 2  4  1 72  0  6]
 [ 6  6  4  0 76  2]
 [ 4 11  3 12  0 62]]
